# Select Queries

In this document I will describe the interface for performing select queries with `doctable`. 

In [1]:

import pandas as pd
import numpy as np
import typing

import sys
sys.path.append('..')
import doctable

#### Define a demonstration schema

The very first step is to define a table schema that will be appropriate for our examples. This table includes the typical `id` column (the first column, specified by `order=0`), as well as string, integer, and boolean attributes. The object used to specify the schema is called a _container_, and I will use that terminology as we go.

In [2]:
@doctable.table_schema
class Record:
    name: str = doctable.Column(column_args=doctable.ColumnArgs(nullable=False, unique=True))
    age: int = doctable.Column()
    is_old: bool = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0, 
            primary_key=True, 
            autoincrement=True
        ),
    )

core = doctable.ConnectCore.open(target=':memory:', dialect='sqlite', echo=True)

with core.begin_ddl() as ddl:
    rtab = ddl.create_table_if_not_exists(container_type=Record)

2024-05-15 14:48:27,852 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:27,853 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Record")
2024-05-15 14:48:27,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-15 14:48:27,855 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Record")
2024-05-15 14:48:27,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-15 14:48:27,857 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Record" (
	id INTEGER, 
	name VARCHAR NOT NULL, 
	age INTEGER, 
	is_old INTEGER, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2024-05-15 14:48:27,857 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2024-05-15 14:48:27,858 INFO sqlalchemy.engine.Engine COMMIT


#### Insert test data

We insert the test data using the `TableQuery` interface. Because this document is about select queries, feel free to look over this for now. I show the contents of the table as a dataframe below. The interface for doing this will be covered later in this document.

In [3]:
import random
random.seed(0)

new_records: typing.List[Record] = list()
for i in range(10):
    age = int(random.random()*100) # number in [0,1]
    is_old = age > 50
    new_records.append(Record(name='user_'+str(i), age=age, is_old=is_old))

# insert new records
rtab.query().insert_multi(new_records)

# dataframe select (for example purposes - .df() will be covered later)
r = core.query().select(rtab[:]).df()
r

2024-05-15 14:48:27,867 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:27,868 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (name, age, is_old) VALUES (?, ?, ?)
2024-05-15 14:48:27,869 INFO sqlalchemy.engine.Engine [generated in 0.00194s] [('user_0', 84, True), ('user_1', 75, True), ('user_2', 42, False), ('user_3', 25, False), ('user_4', 51, True), ('user_5', 40, False), ('user_6', 78, True), ('user_7', 30, False), ('user_8', 47, False), ('user_9', 58, True)]
2024-05-15 14:48:27,871 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:27,872 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
2024-05-15 14:48:27,872 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1
2,3,user_2,42,0
3,4,user_3,25,0
4,5,user_4,51,1
5,6,user_5,40,0
6,7,user_6,78,1
7,8,user_7,30,0
8,9,user_8,47,0
9,10,user_9,58,1


## Working With `DBTable` Objects

In [4]:
print(type(rtab))

<class 'doctable.dbtable.dbtable.DBTable'>


### Selecting Specific Columns

In many cases, you would not want to select all columns from a given table - for this reason, there are several methods you can use to specify the desired columns. In addition to `table[:]` in the above snippet, you may use any of these methods.

| Method | Description |
| --- | --- |
| `table[:]` | Returns list of all columns. |
| `table['col1']` | Returns a single column object. |
| `table['col1',]` | Returns a single column object inside a list. |
| `table['col1', 'col2']` | Returns list of specified columns. |
| `table[['col1', 'col2']]` | Returns list of specified columns. |
| `table[('col1', 'col2')]` | Returns list of specified columns. |
| `table['col1':'col3']` | Returns sequential range of columns as a list. |
| `.all_cols()` | DEPRICATED. Returns list of all columns. |
| `.cols('col1', 'col2')` | DEPRICATED. Returns list of specified columns. |

In [5]:
# all columns
rtab[:]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('name', String(), table=<Record>, nullable=False),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>)]

In [6]:
# single-column subscript
rtab['age']

Column('age', Integer(), table=<Record>)

In [7]:
# get a single column inside a list
rtab['age',]

[Column('age', Integer(), table=<Record>)]

In [8]:
# list of columns
rtab[['id','is_old']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('is_old', Integer(), table=<Record>)]

In [9]:
# slice select
rtab['id':'is_old']

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('name', String(), table=<Record>, nullable=False),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>)]

Note that the `.select()` method requires a list of columns, so we can combine these methods by combining the lists they return. Obviously, the order matters for the returned values.

In [10]:
rtab['id','age'] + [rtab['name']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('age', Integer(), table=<Record>),
 Column('name', String(), table=<Record>, nullable=False)]

### Use table as a context manager
Longer queries will often require more references to `DBTable` objects, so creating a shorthand for the table variable is recommended. One way to do this is by using it as a context manager, which simply returns the table itself. While this may be an unusual application of a context manager, I will show later how it can result in queries that are easier to read. The context manager syntax makes it easy to create shorthand variables for multiple tables and for query objects, as I will show later.

In [11]:
with rtab as t:
    print(type(t))

<class 'doctable.dbtable.dbtable.DBTable'>


## Two Query Interfaces: `ConnectQuery` and `TableQuery`

There are two interfaces for performing queries: `ConnectQuery` and `TableQuery`. Select queries can be performed through either interface, and the interface type determines the type that will be returned.

+ **`ConnectQuery`** table-agnostic interface for querying any table in any result format.

+ **`TableQuery`** table-specific interface for querying a specific table. Insert and select from container objects used to define the schema.


In [12]:
# ConnectQuery - table agnostic
with core.query() as q:
    print(type(q))

# TableQuery - queries are relative to specific 
# table, results appear as container objects
with rtab.query() as q:
    print(type(q))

<class 'doctable.query.connectquery.ConnectQuery'>
<class 'doctable.query.tablequery.TableQuery'>


### Use Context Managers
For any but the simplest queries, I recommend using the query interfaces as context managers. Note that you can combine table context managers and query context managers in a single line. This allows us to create much more compact query expressions

In [13]:
with rtab as t, core.query() as q:
    q.select(t[:])

2024-05-15 14:48:28,000 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,001 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,001 INFO sqlalchemy.engine.Engine [cached since 0.1305s ago] ()
2024-05-15 14:48:28,002 INFO sqlalchemy.engine.Engine COMMIT


### `ConnectQuery` Basics

First I will discuss the `ConnectQuery` interface, which is created via the `ConnectCore.query()` method. This object maintains a database connection, and, when used as a context manager, will commit all changes upon exit. It is fine to use the `ConnectQuery` object without a context manager for queries that do not require commits.

This example is the most basic select query we can execute. Note that `ConnectQuery` methods are table-agnostic, so we must specify columns to be selected - in this case, we provide `rtab[:]` to specify that we want to query all columns from the `Record` table. It returns a `sqlalchemy.CursorResult` object that we will discuss later.

In [14]:
core.query().select(rtab[:])

2024-05-15 14:48:28,015 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,016 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,016 INFO sqlalchemy.engine.Engine [cached since 0.1455s ago] ()


In [15]:
with rtab as t, core.query() as q:
    q.select(t[:])

2024-05-15 14:48:28,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,028 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,029 INFO sqlalchemy.engine.Engine [cached since 0.1577s ago] ()
2024-05-15 14:48:28,030 INFO sqlalchemy.engine.Engine COMMIT


The `.select()` method always accepts a list of columns, so be sure to wrap single-column selections in a list.

In [16]:
core.query().select(rtab['name',])

2024-05-15 14:48:28,039 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,040 INFO sqlalchemy.engine.Engine SELECT "Record".name 
FROM "Record"
2024-05-15 14:48:28,040 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()


In [17]:
core.query().select(rtab['id','is_old'])

2024-05-15 14:48:28,051 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,052 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,052 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ()


### Working with Query Results

Now we turn to working with the results objects. So far I have demonstrated values for returning `sqlalchemy.CursorResult` objects, but additional methods are required to return the results in a usable format. The following methods are available for various purposes:

| Method | Description |
| --- | --- |
| `result.all()` | return all results in query
| `result.df()` | return multiple results as a dataframe
| `result.first()` | return first result in query
| `result.one()` | return exactly one result in query. NOTE: raises exception if not exactly one result.
| `result.scalar_one()` | return single result, end query. NOTE: raises exception if not exactly one result.
| `result.scalars().all()` | return single column of results

In [18]:
core.query().select(rtab[:], limit=3).all()

2024-05-15 14:48:28,063 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,064 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,064 INFO sqlalchemy.engine.Engine [generated in 0.00151s] (3, 0)


[(1, 'user_0', 84, 1), (2, 'user_1', 75, 1), (3, 'user_2', 42, 0)]

In [19]:
core.query().select(rtab[:], limit=3).df()

2024-05-15 14:48:28,074 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,075 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,076 INFO sqlalchemy.engine.Engine [cached since 0.01282s ago] (3, 0)


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1
2,3,user_2,42,0


In [20]:
core.query().select(rtab[:]).first()

2024-05-15 14:48:28,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,090 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,090 INFO sqlalchemy.engine.Engine [cached since 0.2191s ago] ()


(1, 'user_0', 84, 1)

In [21]:
# raises exception if more than one result is returned 
# (here I forced this with limit=1)
core.query().select(rtab[:], limit=1).one()

2024-05-15 14:48:28,099 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,100 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,100 INFO sqlalchemy.engine.Engine [cached since 0.03746s ago] (1, 0)


(1, 'user_0', 84, 1)

In [22]:
# this returns the first column from the first row, then closes the cursor
core.query().select(rtab[:], limit=1).scalar_one()

2024-05-15 14:48:28,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,123 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,124 INFO sqlalchemy.engine.Engine [cached since 0.06108s ago] (1, 0)


1

In [23]:
# it makes more sense to query a single column
core.query().select(rtab['is_old',], limit=1).scalar_one()

2024-05-15 14:48:28,141 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,142 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,142 INFO sqlalchemy.engine.Engine [generated in 0.00130s] (1, 0)


1

In [24]:
# and when when you need a single column, use .scalars() instead of .all()
core.query().select(rtab['is_old',]).scalars().all()

2024-05-15 14:48:28,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,156 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
2024-05-15 14:48:28,156 INFO sqlalchemy.engine.Engine [generated in 0.00177s] ()


[1, 1, 0, 0, 1, 0, 1, 0, 0, 1]

### Conditional Select Statements

| operator | description |
| --- | --- |
| `&`, `doctable.exp.and_()` | and |
| `\|`, `doctable.exp.or_()` | or |
| `==` | equals |
| `!=`, `doctable.exp.not_()` | not equals |
| `>` | greater than |
| `>=` | greater than or equal to |
| `<` | less than |
| `<=` | less than or equal to |
| `in_()` | in list |
| `contains()` | check if item is substring |
| `like()` | like string |
| `ilike()` | case-insensitive like string |
| `between()`, `doctable.exp.between()` | between two values |
| `is_()` | is value |
| `isnot()` | is not value |
| `startswith()` | starts with string |

In [25]:
core.query().select(rtab[:], where=rtab['id']==2).df()

2024-05-15 14:48:28,167 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,168 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id = ?
2024-05-15 14:48:28,169 INFO sqlalchemy.engine.Engine [generated in 0.00170s] (2,)


,id,name,age,is_old
0,2,user_1,75,1


In [26]:
with rtab as t, core.query() as q:
    df = q.select(t[:], where=t['id'] < t['id']).df()
df

2024-05-15 14:48:28,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,185 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id < "Record".id
2024-05-15 14:48:28,185 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()
2024-05-15 14:48:28,187 INFO sqlalchemy.engine.Engine COMMIT


""


In [27]:
with rtab as t, core.query() as q:
    df = q.select(t[:], where=(t['id'] % 2)==0).df()
df

2024-05-15 14:48:28,199 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,200 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id % ? = ?
2024-05-15 14:48:28,201 INFO sqlalchemy.engine.Engine [generated in 0.00135s] (2, 0)
2024-05-15 14:48:28,202 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,2,user_1,75,1
1,4,user_3,25,0
2,6,user_5,40,0
3,8,user_7,30,0
4,10,user_9,58,1


In [28]:
with rtab as t, core.query() as q:
    condition = (t['id']>=2) & (t['id']<=4) & (t['name']!='user_2')
    df = q.select(t[:], where=condition).df()
df

2024-05-15 14:48:28,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,219 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id >= ? AND "Record".id <= ? AND "Record".name != ?
2024-05-15 14:48:28,220 INFO sqlalchemy.engine.Engine [generated in 0.00177s] (2, 4, 'user_2')
2024-05-15 14:48:28,221 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,2,user_1,75,1
1,4,user_3,25,0


In [29]:
with rtab as t, core.query() as q:
    condition = t['name'].in_(('user_2','user_3'))
    df = q.select(t[:], where=condition).df()
df

2024-05-15 14:48:28,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,240 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".name IN (?, ?)
2024-05-15 14:48:28,240 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ('user_2', 'user_3')
2024-05-15 14:48:28,242 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,3,user_2,42,0
1,4,user_3,25,0


In [30]:
with rtab as t, core.query() as q:
    df = q.select(t[:], where=t['id'].between(2,4)).df()
df

2024-05-15 14:48:28,256 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,257 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id BETWEEN ? AND ?
2024-05-15 14:48:28,257 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (2, 4)
2024-05-15 14:48:28,259 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,2,user_1,75,1
1,3,user_2,42,0
2,4,user_3,25,0


In [31]:
with rtab as t, core.query() as q:
    condition = ~(t['name'].in_(('user_2','user_3'))) & (t['id'] < 4)
    df = q.select(t[:], where=condition).df()
df

2024-05-15 14:48:28,276 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,276 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE ("Record".name NOT IN (?, ?)) AND "Record".id < ?
2024-05-15 14:48:28,277 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ('user_2', 'user_3', 4)
2024-05-15 14:48:28,279 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1


In [32]:
with rtab as t, core.query() as q:
    condition = doctable.exp.or_(doctable.exp.not_(t['id']==4)) & (t['id'] <= 2)
    df = q.select(t[:], where=condition).df()
df

2024-05-15 14:48:28,292 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,293 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".id != ? AND "Record".id <= ?
2024-05-15 14:48:28,294 INFO sqlalchemy.engine.Engine [generated in 0.00146s] (4, 2)
2024-05-15 14:48:28,295 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1


In [33]:
with rtab as t, core.query() as q:
    ages = q.select([t['age']]).scalars().all()
    mean_age = sum(ages)/len(ages)
    result = q.select(t[:], where=t['age']>mean_age)
result.df()

2024-05-15 14:48:28,308 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,309 INFO sqlalchemy.engine.Engine SELECT "Record".age 
FROM "Record"
2024-05-15 14:48:28,309 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()
2024-05-15 14:48:28,311 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,311 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (53.0,)
2024-05-15 14:48:28,312 INFO sqlalchemy.engine.Engine COMMIT


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1
2,7,user_6,78,1
3,10,user_9,58,1


### Column Operators

In addition to any of the methods used for conditional selects, there are several additional methods that can be used to transform columns in the select statement.

| Method | Description |
| --- | --- |
| `.label()` | rename column in result (particularly useful after transformations) |
| `.min()`, `doctable.f.min()` | max of column values |
| `.max()`, `doctable.f.max()` | max of column values |
| `.sum()`, `doctable.f.sum()` | sum of column |
| `.count()`, `doctable.f.count()` | count number of results. *NOTE*: must use `f.count()` when counting transformed columns. |
| `.distinct()`, `doctable.f.distinct()` | get distinct values |
| `/` | divide |
| `*` | multiply |
| `+` | add |
| `-` | subtract |
| `%` | modulo |
| `.concat()` | concatenate strings |

In [34]:
columns = [
    (rtab['id'] % 2).label('mod_id'), 
    rtab['name'].label('myname')
]
core.query().select(columns, where=rtab['is_old']).df()

2024-05-15 14:48:28,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,337 INFO sqlalchemy.engine.Engine SELECT "Record".id % ? AS mod_id, "Record".name AS myname 
FROM "Record" 
WHERE "Record".is_old
2024-05-15 14:48:28,337 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (2,)


,mod_id,myname
0,1,user_0
1,0,user_1
2,1,user_4
3,1,user_6
4,0,user_9


In [35]:
formula = rtab['age'].sum() / rtab['age'].count()
core.query().select([formula]).scalar_one()

2024-05-15 14:48:28,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,351 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record"
2024-05-15 14:48:28,352 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ()


Decimal('53.0000000000')

In [36]:
formula = rtab['age'].max() - rtab['age'].min()
core.query().select([formula]).scalar_one()

2024-05-15 14:48:28,362 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,364 INFO sqlalchemy.engine.Engine SELECT max("Record".age) - min("Record".age) AS anon_1 
FROM "Record"
2024-05-15 14:48:28,364 INFO sqlalchemy.engine.Engine [generated in 0.00165s] ()


59

In [37]:
# average age of individuals over 30
formula = rtab['age'].sum() / rtab['age'].count()
condition = rtab['age'] > 30
core.query().select([formula], where=condition).scalar_one()

2024-05-15 14:48:28,375 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,376 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,376 INFO sqlalchemy.engine.Engine [generated in 0.00117s] (30,)


Decimal('59.3750000000')

In [38]:
# descriptive stats on age of individuals over 30
columns = [
    (rtab['age'].sum() / rtab['age'].count()).label('mean'),
    rtab['age'].max().label('max'),
    rtab['age'].min().label('min'),
]
condition = rtab['age'] > 30
core.query().select(columns, where=condition).df()

2024-05-15 14:48:28,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,388 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS mean, max("Record".age) AS max, min("Record".age) AS min 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,388 INFO sqlalchemy.engine.Engine [generated in 0.00176s] (30,)


,mean,max,min
0,59.3750000000,84,40


In [39]:
# all distinct values
formula = rtab['is_old'].distinct()
core.query().select([formula]).df()

2024-05-15 14:48:28,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,406 INFO sqlalchemy.engine.Engine SELECT distinct("Record".is_old) AS distinct_1 
FROM "Record"
2024-05-15 14:48:28,407 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ()


,distinct_1
0,1
1,0


In [40]:
# count individuals over 30
core.query().select([doctable.f.count()], where=rtab['age']>30).scalar_one()

2024-05-15 14:48:28,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,419 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,420 INFO sqlalchemy.engine.Engine [generated in 0.00125s] (30,)


8

In [41]:
# similar to previous
core.query().select([rtab['id'].count()], where=rtab['age']>30).scalar_one()

2024-05-15 14:48:28,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,435 INFO sqlalchemy.engine.Engine SELECT count("Record".id) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,435 INFO sqlalchemy.engine.Engine [generated in 0.00151s] (30,)


8

### Additional Parameters: Order By, Group By, Limit, Offset

More complicated queries involving ordering, grouping, limiting, and specifying offset can be specified using parameters to the `.select()` method.

| Parameter | Description |
| --- | --- |
| `limit` | limit number of results |
| `order_by` | list of columns to order by |
| `group_by` | list of columns to group by |
| `offset` | offset results by specified number |

#### Order By and Limits

In [42]:
# get the five youngest individuals in order
core.query().select(rtab[:], order_by=rtab['age'], limit=5).df()

2024-05-15 14:48:28,445 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,446 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" ORDER BY "Record".age
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,446 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (5, 0)


,id,name,age,is_old
0,4,user_3,25,0
1,8,user_7,30,0
2,6,user_5,40,0
3,3,user_2,42,0
4,9,user_8,47,0


In [43]:
# get the five oldest now
core.query().select(rtab.all_cols(), order_by=rtab['age'].desc(), limit=5).df()

2024-05-15 14:48:28,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,461 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" ORDER BY "Record".age DESC
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,462 INFO sqlalchemy.engine.Engine [generated in 0.00170s] (5, 0)


,id,name,age,is_old
0,1,user_0,84,1
1,7,user_6,78,1
2,2,user_1,75,1
3,10,user_9,58,1
4,5,user_4,51,1


In [44]:
# order by is_old, but preserve order of id otherwise
order = [
    rtab['is_old'].desc(),
    rtab['id'].asc(),
]
core.query().select(rtab[:], order_by=order, limit=5).df()

2024-05-15 14:48:28,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,476 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" ORDER BY "Record".is_old DESC, "Record".id ASC
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,476 INFO sqlalchemy.engine.Engine [generated in 0.00129s] (5, 0)


,id,name,age,is_old
0,1,user_0,84,1
1,2,user_1,75,1
2,5,user_4,51,1
3,7,user_6,78,1
4,10,user_9,58,1


#### Grouping and Column Operators

In [45]:
# summary stats by is_old
cols = [
    #rtab['is_old'].count().label('count'),
    doctable.f.count().label('count'),
    rtab['age'].min().label('min'),
    rtab['age'].max().label('max'),
    (rtab['age'].sum()/rtab['age'].count()).label('mean'),
]
core.query().select(cols, group_by=rtab['is_old']).df()

2024-05-15 14:48:28,497 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,498 INFO sqlalchemy.engine.Engine SELECT count(*) AS count, min("Record".age) AS min, max("Record".age) AS max, sum("Record".age) / (count("Record".age) + 0.0) AS mean 
FROM "Record" GROUP BY "Record".is_old
2024-05-15 14:48:28,498 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ()


,count,min,max,mean
0,5,25,47,36.8000000000
1,5,51,84,69.2000000000


In [46]:
# summarize age by decade
decade_expression = doctable.f.round(rtab['age'] / 10)
cols = [
    decade_expression.label('decade'),
    rtab['age'].count().label('count'),
    rtab['age'].min().label('min'),
    rtab['age'].max().label('max'),
    (rtab['age'].sum()/rtab['age'].count()).label('mean'),
]
core.query().select(cols, group_by=decade_expression).df()

2024-05-15 14:48:28,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,513 INFO sqlalchemy.engine.Engine SELECT round("Record".age / (? + 0.0)) AS decade, count("Record".age) AS count, min("Record".age) AS min, max("Record".age) AS max, sum("Record".age) / (count("Record".age) + 0.0) AS mean 
FROM "Record" GROUP BY round("Record".age / (? + 0.0))
2024-05-15 14:48:28,514 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (10, 10)


,decade,count,min,max,mean
0,3.0,2,25,30,27.5000000000
1,4.0,2,40,42,41.0000000000
2,5.0,2,47,51,49.0000000000
3,6.0,1,58,58,58.0000000000
4,8.0,3,75,84,79.0000000000


#### Offset and Selecting Chunks

The `offset` parameter is used to pagify results into multiple queries - something that is particularly useful if the result set is too larget to fit into memory.

In [47]:
# get the three oldest individuals, offset by three
core.query().select(rtab.all_cols(), order_by=rtab['age'].desc(), limit=3, offset=3).df()

2024-05-15 14:48:28,529 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,530 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" ORDER BY "Record".age DESC
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,530 INFO sqlalchemy.engine.Engine [generated in 0.00120s] (3, 3)


,id,name,age,is_old
0,10,user_9,58,1
1,5,user_4,51,1
2,9,user_8,47,0


In [48]:
for chunk in core.query().select_chunks(rtab[:], chunksize=3):
    print(chunk)

2024-05-15 14:48:28,542 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,543 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,544 INFO sqlalchemy.engine.Engine [generated in 0.00134s] (3, 0)
[(1, 'user_0', 84, 1), (2, 'user_1', 75, 1), (3, 'user_2', 42, 0)]
2024-05-15 14:48:28,545 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,545 INFO sqlalchemy.engine.Engine [cached since 0.002909s ago] (3, 3)
[(4, 'user_3', 25, 0), (5, 'user_4', 51, 1), (6, 'user_5', 40, 0)]
2024-05-15 14:48:28,546 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,547 INFO sqlalchemy.engine.Engine [cached since 0.004557s ago] (3, 6)
[(7, 'user_6', 78, 1), (8, 'user_7', 30, 0), (9, 'user_8', 47, 0)]
2024-05

## `TableQuery` Basics

The `TableQuery` interface is used to make table-specific queries, and, in exchange for this restriction, allows you to insert and select container objects directly. Queries on tables look much like their table-agnostic counterparts, with a few exceptions. Every query still begins with the `.query()` method, which returns a `TableQuery` object with methods for inserting and selecting container objects.

1. List of selected columns is optional - if you do not specify, the query will default to all columns in the table. Otherwise, you should provide a subset of the columns, where all attributes that were not received will refer to `doctable.MISSING`, which you may check for downstream.

2. Results of a select query are returned as a list of container objects. This means that we have called the `.all()` method on the `sqlalchemy.CursorResult` object.

3. All returned results must match the attributes of the container object. Most often you will want to select raw database rows, but transformations via `group_by` and other operators are also possible as long as the result set attributes match the container attributes - that is, they can be expanded to the container constructor.

4. Behavior of `where`, `order_by`, `limit`, are `offset` all operate as-expected.

Below you can see a few examples demonstrating this behavior.

In [49]:
rtab.query().select(where=rtab['age']>50)

2024-05-15 14:48:28,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,599 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record" 
WHERE "Record".age > ?
2024-05-15 14:48:28,600 INFO sqlalchemy.engine.Engine [generated in 0.00176s] (50,)


[Record(name='user_0', age=84, is_old=1, id=1),
 Record(name='user_1', age=75, is_old=1, id=2),
 Record(name='user_4', age=51, is_old=1, id=5),
 Record(name='user_6', age=78, is_old=1, id=7),
 Record(name='user_9', age=58, is_old=1, id=10)]

In [50]:
result = rtab.query().select(rtab.cols('id', 'age'), where=rtab['is_old'])
f'{result[0].name is doctable.MISSING=}', result

2024-05-15 14:48:28,609 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,610 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age 
FROM "Record" 
WHERE "Record".is_old
2024-05-15 14:48:28,611 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ()


('result[0].name is doctable.MISSING=True',
 [Record(name=MISSING, age=84, is_old=MISSING, id=1),
  Record(name=MISSING, age=75, is_old=MISSING, id=2),
  Record(name=MISSING, age=51, is_old=MISSING, id=5),
  Record(name=MISSING, age=78, is_old=MISSING, id=7),
  Record(name=MISSING, age=58, is_old=MISSING, id=10)])

In [51]:
# this is valid, although perhaps not recommended
cols = [
    (rtab['age'].sum()/rtab['age'].count()).label('age'),
]
rtab.query().select(cols=cols)

2024-05-15 14:48:28,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,624 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS age 
FROM "Record"
2024-05-15 14:48:28,625 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()


[Record(name=MISSING, age=Decimal('53.0000000000'), is_old=MISSING, id=MISSING)]

`.select_chunks()` also more or less works as-expected, with the chunks being converted to container objects.

In [52]:
for chunk in rtab.query().select_chunks(chunksize=3):
    print(chunk)

2024-05-15 14:48:28,636 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:48:28,637 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,637 INFO sqlalchemy.engine.Engine [cached since 0.09464s ago] (3, 0)
[Record(name='user_0', age=84, is_old=1, id=1), Record(name='user_1', age=75, is_old=1, id=2), Record(name='user_2', age=42, is_old=0, id=3)]
2024-05-15 14:48:28,638 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 14:48:28,639 INFO sqlalchemy.engine.Engine [cached since 0.09626s ago] (3, 3)
[Record(name='user_3', age=25, is_old=0, id=4), Record(name='user_4', age=51, is_old=1, id=5), Record(name='user_5', age=40, is_old=0, id=6)]
2024-05-15 14:48:28,640 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".name, "Record".age, "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2024-05-15 